<div class="alert alert-success" style = "border-radius: 20px;text-align: center;" role="alert">
    Don't forget to upvote if you liked the notebook
</div>

# Importing Libraries

In [ ]:
!pip show torch

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import PIL

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Preparing directories

In [ ]:
# Path
img_dir = '/content/drive/MyDrive/kaggle/input/face-mask-detection/images'
"""
annotation_directory contains files, and each file is associated to only one image,
and it contains the height and width of the image and also xmin,ymin,xmax, and ymax of each boundary box
inside the image
"""
annotation_dir = '/content/drive/MyDrive/kaggle/input/face-mask-detection/annotations'

input_dir = '/content/drive/MyDrive/kaggle/input/face-mask-detection'
output_dir = '/content/drive/MyDrive/kaggle/working'

# Viewing some images

In [ ]:
# sample visuaization
for idx, image in enumerate(os.listdir(img_dir)):
    img = cv2.imread(os.path.join(img_dir, image), 1)
    plt.imshow(img)
    plt.show()

    if idx == 3:
        break

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Storing images paths

In [ ]:
img_file_path = []
for img in os.listdir(img_dir): # img here is the name of the image not the image itself
    image = cv2.imread(os.path.join(img_dir,img), 0) # 0 for grayscale
    img_file_path.append(f'{img}')

# Libraries needed for reading XML files

In [ ]:
import xml.etree.ElementTree as ET
"""
ElementTree module provides
a way to work with Extensible Markup Language (XML) documents as a tree-like structure of elements.
"""
import glob

# Reading XML files

In [ ]:
df = { 'name': [],
        'label': [],
      'width': [],
      'height': [],
     'xmin': [],
     'ymin': [],
     'xmax': [],
     'ymax': []}
"""
The glob.glob() function returns a list of all the pathnames that match the specified pattern.
The resulting list can then be used to process or analyze the files that match the pattern.
"""
for idx, anno in enumerate(glob.glob(annotation_dir + '/*.xml')):
    trees = ET.parse(anno)

    #print(anno) print/view the annotation to understand the following code
    root = trees.getroot()
    width, height = [], []
    for item in root.iter():
        if item.tag == 'size':
            for attr in list(item):
                if attr.tag == 'width':
                    width = int(round(float(attr.text)))
                if attr.tag == 'height':
                    height = int(round(float(attr.text)))

        if item.tag == 'object':
            for attr in list(item):
                if 'name' in attr.tag:
                    label = attr.text
                    df['label'] += [label]
                    df['width'] += [width]
                    df['height'] += [height]
                    #dataset['name']+=[anno.split('/')[-1][0:-4]]
                    df['name'] += [anno.split('/')[-1][0:-4]]

                if 'bndbox' in attr.tag:
                    for dim in attr:
                        if dim.tag == 'xmin':
                            xmin = int(round(float(dim.text)))
                            df['xmin'] += [xmin]

                        if dim.tag == 'ymin':
                            ymin = int(round(float(dim.text)))
                            df['ymin'] += [ymin]
                        if dim.tag == 'xmax':
                            xmax = int(round(float(dim.text)))
                            df['xmax'] += [xmax]
                        if dim.tag == 'ymax':
                            ymax = int(round(float(dim.text)))
                            df['ymax'] += [ymax]



# Viewing data frame of images

In [ ]:
df1 = pd.DataFrame(df)
df1.head()

# Describing data frame

In [ ]:
df1.info()

# Maping labels to integers

In [ ]:
label_map = { 'without_mask': 0,
            'with_mask	': 1}

df1['class'] = df1['label'].map(label_map)

In [ ]:
df1.head()

# Splitting the data

In [ ]:
# split train, test, val data
from sklearn.model_selection import train_test_split

train, test = train_test_split(img_file_path, test_size=0.2, random_state=101)
train, val = train_test_split(train, test_size=0.15, random_state=101)

# Preparing yolo v5 model

In [ ]:
# yolo v5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -qr requirements.txt

# Preparing some new directories

In [ ]:
# create a separate folder structure for train, validation and test image and validation files
os.chdir('/content/drive/MyDrive/kaggle/working')
"""
The os.chdir() function is used to change the working directory
of the Python script to the specified path.
"""
os.mkdir('/content/yolov5/data/train')
os.mkdir('/content/yolov5/data/val')
os.mkdir('/content/yolov5/data/test')
os.mkdir('/content/yolov5/data/train/images')
os.mkdir('/content/yolov5/data/train/labels')
os.mkdir('/content/yolov5/data/val/images')
os.mkdir('/content/yolov5/data/val/labels')
os.mkdir('/content/yolov5/data/test/images')
os.mkdir('/content/yolov5/data/test/labels')

# Copying the image data in the yolov5 folder

In [ ]:
def open_image_file(image_items, folder_name):

    for image in image_items:
        img = PIL.Image.open(f'{img_dir}/{image}')
        img1 = img.resize((640, 480))
        _ = img1.save(f'/content/yolov5/data/{folder_name}/images/{image}')

open_image_file(train, 'train')
open_image_file(val, 'val')
open_image_file(test, 'test')

# Resizing boxes to match with the new images size

In [ ]:
df1['xmin'] = (640/df1['width']) * df1['xmin']
df1['ymin'] = (480/df1['height']) * df1['ymin']
df1['xmax'] = (640/df1['width']) * df1['xmax']
df1['ymax'] = (480/df1['height']) * df1['ymax']
df1.head()

In [ ]:
df1[['xmin', 'ymin', 'xmax', 'ymax']] = df1[['xmin', 'ymin', 'xmax', 'ymax']].astype('int')

In [ ]:
WIDTH = 640
HEIGHT = 480

# Converting from left upper corner and right bottom corner of boxes to just midpoint, height, and width of boxes

In [ ]:
df1['x_center'] = (df1['xmin']+df1['xmax'])/(2*WIDTH)
df1['y_center'] = (df1['ymin']+df1['ymax'])/(2*HEIGHT)
df1['box_width'] = (df1['xmax']-df1['xmin'])/ WIDTH
df1['box_height'] = (df1['ymax']-df1['ymin'])/ HEIGHT

In [ ]:
df1.head()

In [ ]:
df1 = df1.astype(str)

# Viewing some images paths

In [ ]:
img_file_path[:5]

# Writing info for each images

In [ ]:
def copy_label(label_items, folder_name):
    file_name = [x.split('.')[0] for x in img_file_path]
    for name in file_name:
        data = df1[df1.name == name]
        box_list = []
        for idx in range(len(data)):
            row = data.iloc[idx]
            box_list.append(row['class']+" "+row['x_center']+" "+row['y_center']+" "+ row['box_width']+" "+row['box_height'])

        text = "\n".join(box_list)
        with open(f'/content/yolov5/data/{folder_name}/labels/{name}.txt', 'w') as file:
            file.write(text)

In [ ]:
copy_label(train, 'train')
copy_label(val, 'val')
copy_label(test, 'test')

In [ ]:
os.chdir('/content/yolov5/data/train/labels')

In [ ]:
cat maksssksksss0.txt

# Creating yaml file

In [ ]:
# Configure .yaml file
yaml_file = """train: /content/yolov5/data/train/images
val: /content/yolov5/data/val/images

nc: 2
names: [without_mask, with_mask]"""

with open('/content/yolov5/data/data.yaml', 'w') as f:
    f.write(yaml_file)

In [ ]:
%cat /content/yolov5/data/data.yaml

# Training

In [ ]:
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 50 --data  /content/yolov5/data/data.yaml --cfg  /content/yolov5/models/yolov5s.yaml --weights '' --name best


# Displaying Training and Validation batches

In [ ]:
!python /content/yolov5/detect.py --source /content/drive/MyDrive/kaggle/input/face-mask-detection/sample2.jpg --weights /content/drive/MyDrive/workk/yolov5/runs/train/best/weights/best.pt --img-size 640


In [ ]:
import torch
import cv2
import numpy as np

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/workk/yolov5/runs/train/best/weights/best.pt')

# Define the webcam video stream
cap = cv2.VideoCapture(0)  # 0 for default webcam

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Perform object detection
    results = model(frame)

    # Render the results on the frame
    frame = np.squeeze(results.render())

    # Display the resulting frame
    cv2.imshow('YOLOv5 Detection', frame)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close any OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
!python --version